In [1]:
import FinanceDataReader as fdr
import time
from collections import deque
import datetime
import pandas as pd
import filter
import atr
import Long_order
import pandas_market_calendars as mcal # 영업일 계산 모듈

# 매게변수 ticker 추가 
def get_balance_coin(balances, ticker):
    bool = True
    while True:
        # for문 추가
        for i in range(1, 11):
            if ticker in balances[i]['currency']:
                balance = balances[i]['balance']
                avg_buy_price = balances[i]['avg_buy_price']
                value = (float(avg_buy_price), float(balance))
                bal_loc = int(i)
                bool = False
                break
        if bool == False:
            break
        else:
            pass
        for j in range(1, 11):
            if balances[j]['currency'] == '':
                value = (int(0), int(0))
                bal_loc = int(j)
                bool == False
                break
        if bool == False:
            break
        for k in range(1, 11):
            if balances[j]['currency'] != '':
                value = (str('False'), str('False'))
                bal_loc = str('False')
                break
        break
    return value, bal_loc

# 매게변수 ticker 추가
def get_ticker_ma(volume, df_ticker, balances, ticker, atr_value):

    now_price = df_ticker.iloc[-1]['Close']                     # 코인 현재가
    coin_check = get_balance_coin(balances, ticker) # 코인 보유 하고 있는지 체크
    avg_price = coin_check[0][0]                         # 매수 평균가
    balance = coin_check[0][1]                             # 코인 보유 개수
    # 코인의 지갑 위치값 추가
    bal_loc = coin_check[1]
    if type(avg_price) == str:
        return
    print("지갑 위치 = {}".format(bal_loc))
    # 손절매 가격
    sell_price = avg_price - (5 * atr_value)
    # 만약 현재가격이 평균가 보다 높아지면 손절매의 가격을 높인다.
    if now_price > avg_price:
        sell_price = now_price - (5 * atr_value)
    # 매수 평균가가 int 이면 매수 조건 체크 float이면 매도 조건 체크
    if type(avg_price) == int:

        # 살 가격 정하기
        # 총자산의 10퍼
        buy_price = float(balances[0]['balance']) / empty_count
        Long_order.long_buy_order(buy_price, balances, bal_loc, ticker, df_ticker)
    else:
        # 현재 보유 코인 수익률 계산
        buy_profit = ((now_price - avg_price)/avg_price) * 100
        profit = round(buy_profit, 2)
        if avg_price < sell_price:
            Long_order.long_sell_order(balances, bal_loc, ticker, df_ticker)
        else:
            print(f"코인명: {ticker}, 수익률: {profit}%")
            print("내가산 가격 = {}, 시장가격 = {}".format(avg_price, now_price))
            print('---------------------------------------------')
i = 0
tickers = []
ma10 = []
ma25 = []
ma50 = []
balances = []
# 시작 날짜 입력
day = datetime.datetime(2020,4,15)
# 초기 티커 필터
endday_change = '{}-{}-{}'.format(day.year, day.month, day.day)
# 초기 지갑 잔고 입력
volume = 10000

balances = ({'currency': 'dolor', 'balance': '{}'.format(volume), 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''} #1
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''} # 5
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''} # 9 실제 딕셔너리 주소
, {'currency': '', 'balance': '', 'avg_buy_price': ''})
empty_count = 9
assets_total = []
day_total = []
while True:
    assets = 0
    while True:
        endday_change = '{}-{}-{}'.format(day.year, day.month, day.day)
        if len(mcal.get_calendar('NYSE').schedule(start_date=endday_change, end_date=endday_change)) == 0:
            print('{} 영업일 아님'.format(endday_change))
            break
        print('---------------------------------------------')
        print(day)
        startday = day - datetime.timedelta(days = 60)
        while True:
            startday_change = '{}-{}-{}'.format(startday.year, startday.month, startday.day)
            endday_change = '{}-{}-{}'.format(day.year, day.month, day.day)
            count = len(mcal.get_calendar('NYSE').schedule(start_date=startday_change, end_date=endday_change))
            if count == 51:
                break
            startday -= datetime.timedelta(days = 1)

        # S&P500 주가를 넣는다
        df = fdr.DataReader(symbol='US500', start = startday_change, end = endday_change)

        ma10 = df['Close'].rolling(window=10).mean()
        df.insert(len(df.columns), "MA10", ma10)

        ma25 = df['Close'].rolling(window=25).mean()
        df.insert(len(df.columns), "MA25", ma25)

        ma50 = df['Close'].rolling(window=50).mean()
        df.insert(len(df.columns), "MA50", ma50)

        print('종가: {:,.0f}    ma10: {:,.0f}     ma25: {:,.0f}    ma50: {:,.0f}'.format(df.iloc[-1]['Close'], df.iloc[-1]['MA10'], df.iloc[-1]['MA25'], df.iloc[-1]['MA50']))

        volume = float(balances[0]['balance'])
        
        for k in range(1, 11):
            empty_count += 1

        if df.iloc[-1]['MA10'] > df.iloc[-1]['MA25'] and df.iloc[-1]['Close'] > df.iloc[-1]['MA50']:
            
            
            for k in range(1,11):
                if balances[k]['currency'] == '':
                    start_filter = True
                    break
                    
            if start_filter == True:
                tickers = filter.filter(day, endday_change)
            start_filter = False

            for i in range(1, 11):
                tickers.append(balances[i]['currency'])
            # ticker 중복값 제거
            tickers = list(set(tickers))
            # 롱 주문
            for j in range(len(tickers)):
                ticker = tickers[j]
                if ticker != '':
                    df_ticker = fdr.DataReader(symbol=ticker, start= startday_change, end=endday_change)
                    atr_value = atr.ATR(day, endday_change, ticker)
                    get_ticker_ma(volume, df_ticker, balances, ticker, atr_value)
                    now_price = df_ticker.iloc[-1]['Close']                     # 코인 현재가
                    coin_check = get_balance_coin(balances, ticker) # 코인 보유 하고 있는지 체크
                    balance = coin_check[0][1]                             # 코인 보유 개수
                    assets += now_price * balance
            empty_count = 0

        elif df.iloc[-1]['MA10'] < df.iloc[-1]['MA25'] or df.iloc[-1]['Close'] < df.iloc[-1]['MA50']:
            # 가지고 있던 코인 팔기
            for j in range(len(tickers)):
                ticker = tickers[j]
                if ticker != '':
                    df_ticker = fdr.DataReader(symbol=ticker, start= startday_change, end=endday_change)
                    now_price = df_ticker.iloc[-1]['Close']
                    coin_check = get_balance_coin(balances, ticker)
                    avg_price = coin_check[0][0]
                    bal_loc = coin_check[1]
                    if type(avg_price) == float:
                        Long_order.long_sell_order(balances, bal_loc, ticker, df_ticker)
            
            print('기다리는중')
        break

    if len(mcal.get_calendar('NYSE').schedule(start_date=endday_change, end_date=endday_change)) != 0:
        assets += float(balances[0]['balance'])
        day_total.append(day)
        assets_total.append(assets)
        print("총 평가 자산 = {:,.0f}달러".format(assets))
        
    # 시간에 24시간씩 계속 추가
    day = day + datetime.timedelta(hours = 24)

    if day == datetime.datetime(2020, 12, 31):
        break
    i += 1
    time.sleep(0.2)


---------------------------------------------
2020-04-15 00:00:00
종가: 2,846    ma10: 2,654     ma25: 2,574    ma50: 2,892
기다리는중
총 평가 자산 = 10,000달러
---------------------------------------------
2020-04-16 00:00:00
종가: 2,783    ma10: 2,674     ma25: 2,570    ma50: 2,882
기다리는중
총 평가 자산 = 10,000달러
---------------------------------------------
2020-04-17 00:00:00
종가: 2,800    ma10: 2,707     ma25: 2,572    ma50: 2,872
기다리는중
총 평가 자산 = 10,000달러
2020-4-18 영업일 아님
2020-4-19 영업일 아님
---------------------------------------------
2020-04-20 00:00:00
종가: 2,875    ma10: 2,742     ma25: 2,588    ma50: 2,863
※ 5달러 이하 종목 삭제 중 ※
HTTP Error 404: Not Found  - symbol "BRKB"not found or invalid periods
HTTP Error 404: Not Found  - symbol "BFB"not found or invalid periods
HTTP Error 400: Bad Request  - symbol "CEG"not found or invalid periods
HTTP Error 400: Bad Request  - symbol "OGN"not found or invalid periods
※ 5달러 이하 종목 삭제 완료 ※
430
※ 평균 거래 금액 5천만달러 이하 삭제 중 ※
※ 평균 거래 금액 5천만달러 이하 삭제 완료 ※
184
※ 변동률 순위 구하는 중 ※

KeyboardInterrupt: 

In [4]:
import datetime
import FinanceDataReader as fdr
import pandas_market_calendars as mcal
import pandas as pd
import filter
day = datetime.datetime(2022,5,1)
endday_change = '{}-{}-{}'.format(day.year, day.month, day.day)
filter.filter(day, endday_change)

※ 5달러 이하 종목 삭제 중 ※
HTTP Error 404: Not Found  - symbol "BRKB"not found or invalid periods
예외발생
HTTP Error 404: Not Found  - symbol "BFB"not found or invalid periods
예외발생
※ 5달러 이하 종목 삭제 완료 ※
409
※ 평균 거래 금액 5천만달러 이하 삭제 중 ※
※ 평균 거래 금액 5천만달러 이하 삭제 완료 ※
90
※ 변동률 순위 구하는 중 ※
※ 변동률 순위 구하기 완료 ※


['PYPL', 'DVN', 'OXY', 'NFLX', 'MOS', 'NUE', 'COP', 'CVX', 'CHTR', 'MRNA']